In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
sns.set()
from datetime import datetime
import datetime
import time
import timestring
import math
matplotlib.rcParams['figure.figsize'] = (14, 8)

In [2]:
df = pd.read_csv('./Data/2017.csv')

In [3]:
#remove rows will null values which is approximately 1.5%
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

DEP_TIME                 80308
DEP_DELAY                80343
DEP_DELAY_NEW            80343
TAXI_OUT                 82145
WHEELS_OFF               82141
WHEELS_ON                84674
TAXI_IN                  84674
ARR_TIME                 84674
ARR_DELAY                95211
ARR_DELAY_NEW            95211
ACTUAL_ELAPSED_TIME      95211
AIR_TIME                 95211
CARRIER_DELAY          4645148
WEATHER_DELAY          4645148
NAS_DELAY              4645148
SECURITY_DELAY         4645148
LATE_AIRCRAFT_DELAY    4645148
dtype: int64

In [4]:
df = df.dropna(subset=['DEP_TIME',
       'DEP_DELAY', 'DEP_DELAY_NEW', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON',
       'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'ARR_DELAY_NEW',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME'])

In [5]:
df = df.fillna(0)

In [6]:
df = df[['MONTH','DAY_OF_MONTH', 'DAY_OF_WEEK', 'OP_CARRIER', 'ORIGIN', 'DEST', 'DEP_TIME',
       'DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY', 'AIR_TIME', 'DISTANCE',
       'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY']]

In [7]:
df.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,ARR_TIME,ARR_DELAY,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,3,21,2,B6,BOS,DCA,1031.0,16.0,14.0,3.0,1154.0,3.0,66.0,399.0,0.0,0.0,0.0,0.0,0.0
1,3,21,2,B6,IAD,BOS,1420.0,-15.0,9.0,6.0,1533.0,-29.0,58.0,413.0,0.0,0.0,0.0,0.0,0.0
2,3,21,2,B6,BOS,IAD,1203.0,-12.0,17.0,5.0,1338.0,-14.0,73.0,413.0,0.0,0.0,0.0,0.0,0.0
3,3,21,2,B6,BOS,PHL,758.0,-6.0,21.0,8.0,925.0,-11.0,58.0,280.0,0.0,0.0,0.0,0.0,0.0
4,3,21,2,B6,FLL,BDL,1041.0,-4.0,11.0,5.0,1327.0,-15.0,150.0,1173.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df = df.reindex()

In [9]:
len(df.DEST.unique())

320

In [10]:
len(df.ORIGIN.unique())

319

## Linear Regression

In [85]:
y = df.ARR_DELAY

In [86]:
type(y)

pandas.core.series.Series

In [87]:
X = df.drop(['ARR_DELAY'], axis=1)
X = X.drop(['DEP_DELAY'], axis=1)

In [88]:
X = X.drop(['OP_CARRIER'], axis=1)
X = X.drop(['ORIGIN'], axis=1)
X = X.drop(['DEST'], axis=1)

In [89]:
y = y.values

In [90]:
X = X.values

In [91]:
y = y.reshape(-1,1)

In [94]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [103]:
X_train.shape

(4463528, 14)

In [104]:
y_train.shape

(4463528, 1)

In [105]:
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [106]:
y_pred = reg_all.predict(X_test)

In [107]:
print("R^2: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

R^2: 0.9533517895588974
Root Mean Squared Error: 9.918862828093879


   # Using Cross Validation

In [10]:
from sklearn.model_selection import cross_val_score

In [109]:
cv_scores = cross_val_score(reg_all, X, y, cv=5)
print(cv_scores)

[0.95981712 0.94418122 0.9637719  0.94921291 0.94193189]


# Linear Regression with LabelBinarizer from SciKit learn

In [11]:
from sklearn.preprocessing import LabelBinarizer

In [12]:
#Apply Label Binarizer (LabelEncoder and OneHotEncoder in one step) to OP_CARRIER
carrier_lb = LabelBinarizer()
origin_lb = LabelBinarizer()
dest_lb = LabelBinarizer()

In [13]:
Xcarrier = carrier_lb.fit_transform(df.OP_CARRIER.values)
Xorigin = origin_lb.fit_transform(df.ORIGIN.values)
Xdest = dest_lb.fit_transform(df.DEST.values)

In [14]:
#now we create a new DataFrame with encoded values to model
#convert the transformed Arrays into DataFrames
df_carrier_enc = pd.DataFrame(Xcarrier, columns=["CARRIER_"+str(int(i)) for i in range(Xcarrier.shape[1])])
df_origin_enc = pd.DataFrame(Xorigin, columns=["ORIGIN_"+str(int(i)) for i in range(Xorigin.shape[1])])
df_des_enc = pd.DataFrame(Xdest, columns=["DEST_"+str(int(i)) for i in range(Xdest.shape[1])])

In [15]:
#new encoded dataframe
df_encoded = pd.concat([df, df_carrier_enc], axis=1)

In [ ]:
df_encoded = pd.concat([df_encoded, df_origin_enc], axis=1)

In [ ]:
df_encoded = pd.concat([df_encoded, df_des_enc], axis=1)

In [25]:
df_encoded.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,...,CARRIER_2,CARRIER_3,CARRIER_4,CARRIER_5,CARRIER_6,CARRIER_7,CARRIER_8,CARRIER_9,CARRIER_10,CARRIER_11
0,3.0,21.0,2.0,B6,BOS,DCA,1031.0,16.0,14.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,21.0,2.0,B6,IAD,BOS,1420.0,-15.0,9.0,6.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,21.0,2.0,B6,BOS,IAD,1203.0,-12.0,17.0,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,21.0,2.0,B6,BOS,PHL,758.0,-6.0,21.0,8.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,21.0,2.0,B6,FLL,BDL,1041.0,-4.0,11.0,5.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df_encoded = df_encoded.dropna()

In [37]:
df_encoded.tail()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_CARRIER,ORIGIN,DEST,DEP_TIME,DEP_DELAY,TAXI_OUT,TAXI_IN,...,CARRIER_2,CARRIER_3,CARRIER_4,CARRIER_5,CARRIER_6,CARRIER_7,CARRIER_8,CARRIER_9,CARRIER_10,CARRIER_11
5579405,2.0,4.0,6.0,WN,ATL,TPA,1101.0,-4.0,18.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5579406,2.0,4.0,6.0,WN,ATL,TPA,1610.0,0.0,9.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5579407,2.0,4.0,6.0,WN,ATL,TPA,938.0,-2.0,16.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5579408,2.0,4.0,6.0,WN,ATL,TPA,1442.0,-3.0,9.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5579409,2.0,4.0,6.0,WN,ATL,TPA,1730.0,-10.0,10.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [38]:
y_enc = df_encoded.ARR_DELAY

In [39]:
X_enc = df_encoded.drop(['ARR_DELAY'], axis=1)
X_enc = X_enc.drop(['DEP_DELAY'], axis=1)
X_enc = X_enc.drop(['OP_CARRIER'], axis=1)
X_enc = X_enc.drop(['ORIGIN'], axis=1)
X_enc = X_enc.drop(['DEST'], axis=1)

y_enc = y_enc.values
X_enc = X_enc.values
y_enc = y_enc.reshape(-1,1)

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size = 0.2, random_state=42)

In [42]:
reg_all = LinearRegression()
reg_all.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [43]:
y_pred = reg_all.predict(X_test)

In [44]:
print("R^2: {}".format(reg_all.score(X_test, y_test)))
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: {}".format(rmse))

R^2: 0.9539631785438748
Root Mean Squared Error: 9.887680440821649


In [45]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(reg_all, X_enc, y_enc, cv=5)
print(cv_scores)

[0.96047127 0.94472886 0.96296107 0.94940663 0.94479483]


# Random Forrest